# Data Mining for NFTs

- Scrap and store data from rarity tool website about trendy NFTs
- Find influencers in NFT and spot if they speak about a particular NFT
- Store information about Google trends

Idea for implementation
- When was the Twitter account created
- sentiment analysis on Twitts
- Volume
- Launch date of the NFT
- ETH value (if low buy more easily)

Packages that need to be installed
- !pip install pytrends
- !pip3 install snscrape

In [1]:
# Data manipulation and analysis library
import pandas as pd
import numpy as np

# Progress bar library
from tqdm import tqdm

# We will use snscrape from github.com/JustAnotherArchivist/snscrape to get data from Twitter
import snscrape.modules.twitter as sntwitter
from datetime import date, timedelta

# We use pytrend to fetch information from Google trend
from pytrends.request import TrendReq

# Scrap and store data from rarity tool website about trendy NFTs

In [6]:
# Import selenium webdriver and configure its options
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")

s=Service('./chromedriver')

driver = webdriver.Chrome(options=options, service=s)

In [7]:
# Get driver for rarity tool website
driver.get('https://rarity.tools/upcoming')

In [8]:
# Wait until page is loaded
delay = 10 # seconds
try:
    myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, "//*[contains(@class,'text-left text-gray-800')]")))
    print("Rarity tool page is ready!")
except TimeoutException:
    print("Loading took longer than {0} seconds".format(delay))

# Find NFT elements
url_elements = driver.find_elements_by_xpath("//*[contains(@class,'text-left text-gray-800')]")

# Show first NFT
url_elements[0].text.split('\n')

Rarity tool page is ready!


/var/folders/4t/5wd2c1y903ncr7r3dmlwskrr0000gn/T/ipykernel_40708/220282747.py:10: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  url_elements = driver.find_elements_by_xpath("//*[contains(@class,'text-left text-gray-800')]")


['Doodle Ape Champs',
 '2D Ape themed NFT avatars',
 'Discord',
 '@doodleapechamps',
 'www.doodleapechamps.art',
 '0.5 SOL',
 '250 Total',
 '6:00 pm (Europe/Berlin)',
 'Yesterday']

In [9]:
# Create a dataframe with NFTs from rarity tool
nft_list = []
for element in url_elements:
    nft_list.append(element.text.split('\n'))

df = pd.DataFrame(nft_list)

# Close Selenium driver
driver.close()

In [10]:
# Clean the name, description, website, volume, and twitter account
df['Name'] = df[0]
df['Description'] = df[1]
df['Website'] = df[4]
df['Twitter Profile'] = df[3].apply(lambda x: x[1:])

In [11]:
# Clean the price tag
def cleanPrice(x):
    if 'eth' in x.lower():
        return float(''.join([i for i in x if i.isnumeric() or i=='.']))
    else:
        return 0
df['Price ETH'] = df[5].apply(cleanPrice)

ValueError: could not convert string to float: '0.0650.09'

# Get data on each Twitter profile

In [14]:
# Using TwitterSearchScraper to scrape data and append tweets to list
profile_list = []

# Use TwitterSearchScraper to scrape data for each NFT
for twitter_name in tqdm(df['Twitter Profile']):
    if sntwitter.TwitterUserScraper.is_valid_username(twitter_name):
        try:
            items = next(sntwitter.TwitterProfileScraper('{0}'.format(twitter_name), isUserId=False).get_items())
        except AttributeError:
            print('Twitter profile {0} not found'.format(twitter_name))
        else:  
            profile_list.append([twitter_name, items.user.created, items.user.description, 
                                 items.user.favouritesCount, items.user.followersCount, 
                                 items.user.friendsCount, items.user.listedCount, items.user.statusesCount, 
                                 items.user.mediaCount, items.user.location, items.user.profileBannerUrl, 
                                 items.user.profileImageUrl])
    else:
        print('Twitter profile {0} is not a valid username'.format(twitter_name))

# Creating a dataframe from the tweets list above
profiles_df = pd.DataFrame(profile_list, columns=['Twitter Profile','Twitter User Created', 'Twitter User Description', 'Twitter User Favourites Count', 
                                                  'Twitter User Followers Count', 'Twitter User Friends Count', 'Twitter User Listed Count', 
                                                  'Twitter User Statuses Count', 'Twitter User Media Count', 'Twitter User Location', 
                                                  'Twitter User Profile Banner Url', 'Twitter User Profile Image Url'])

  0%|                                                   | 0/202 [00:00<?, ?it/s]Error retrieving https://api.twitter.com/2/timeline/profile/1482304140265295880.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&include_tweet_replies=true&userId=1482304140265295880&count=100&ext=mediaStats%2ChighlightedLabel: blocked (429)
4 requests to https://api.twitter.com/2/timeline/profile/1482304140265295880.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_

ScraperException: 4 requests to https://api.twitter.com/2/timeline/profile/1482304140265295880.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&include_tweet_replies=true&userId=1482304140265295880&count=100&ext=mediaStats%2ChighlightedLabel failed, giving up.

In [ ]:
# Merge and save dataframe
nft_df = df[['Twitter Profile', 'Name', 'Description', 'Website', 'Price ETH']].merge(profiles_df, on='Twitter Profile', how='left')

nft_df.to_csv('nft_data.csv', index=False)

# Fetch Tweets for each NFT

In [ ]:
# Creating list to append tweet data
tweets_list = []

# We are interested to get tweets from the last 7 days
today = date.today().strftime("%Y-%m-%d")
last_month = (date.today() - timedelta(days=7)).strftime("%Y-%m-%d")

# Maximum tweets to scrap for each query
max_tweets = 5000

# Use TwitterSearchScraper to scrape data for each NFT
for search_query in tqdm(nft_df['Twitter Profile']):
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(
        '{0} since:{1} until:{2}'.format(search_query, last_month, today)).get_items()):
        if i > max_tweets:
                break
        tweets_list.append([search_query, tweet.date, tweet.id, 
                            tweet.content, tweet.user.username, tweet.user.created, 
                            tweet.user.description, tweet.user.favouritesCount, tweet.user.followersCount, 
                            tweet.user.friendsCount, tweet.user.listedCount, tweet.user.statusesCount, 
                            tweet.user.mediaCount, tweet.user.location, tweet.user.profileBannerUrl, 
                            tweet.user.profileImageUrl])

# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets_list, columns=['Twitter Profile', 'Datetime', 'Tweet Id', 'Text', 'Username',
                                               'User Created', 'User Description', 'User Favourites Count', 
                                               'User Followers Count', 'User Friends Count', 'User Listed Count', 
                                               'User Statuses Count', 'User Media Count', 'User Location', 
                                               'User Profile Banner Url', 'User Profile Image Url'])

In [ ]:
# Control if any of the NFTs has been recently mentionned by an influencer

# Influencer list taken on: 
# - https://coinbound.io/top-nft-influencers/
# - https://itsblockchain.com/top-15-nft-influencers-on-twitter-you-should-follow-right-now/

influencers = ['garyvee', 'DeezeFi', 'farokh', 'mevcollector', 'beaniemax', 'RealmissNFT', 
               'CozomoMedici', 'peruggia_v', 'andrwwang', 'iamDCinvestor', 'punk4156',  'punk6529', 
               'TheTreeverse', 'KennethBosak', 'tsmith', 'Bosslogic', 'NFTLive']

print('We found {0} influencers in the list'.format(len(tweets_df.loc[tweets_df['Twitter Profile'].isin(influencers)])))

In [ ]:
# Store Twitter information
tweets_df.to_csv('tweets.csv', index=False)

# Store information about Google trends

In [ ]:
# Article of reference: https://medium.com/geekculture/easily-gather-google-trends-data-in-python-22219cecd6fc

# hl: host language
# tz: timezone in minutes after UTC (positive only)
# timeout: between 5 and 10 sec if the server is not responding
pytrends = TrendReq(hl='en-US', tz=60, timeout=(5,10))

# keyword list
kw_list = ["Nft"]

# cat: google trend category (https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories)
# timeframe: 12 months ago from today
# geo represents the country in two-digit ISO codes (defaults to worldwide if left blank)
# gprop represents Google properties such as news, images, youtube etc. (defaults to web searches if left blank)
pytrends.build_payload(kw_list, cat=0, timeframe='today 12-m', geo='', gprop='')

# Interest over time
df_interest_over_time = pytrends.interest_over_time()

# Interest by Region
df_interest_by_region = pytrends.interest_by_region()

In [ ]:
# Store current trend
df_interest_over_time.to_csv('google_trend_over_time.csv')
df_interest_by_region.to_csv('google_trend_per_region.csv')